In [2]:
#%%
from Crypto.Util.number import getStrongPrime
from math import gcd
from random import randint
from typing import Tuple
from Crypto.Cipher import AES
from hashlib import sha256

def encrypt(m: bytes, secret_key: int) -> str:
    hash_secret_key = sha256(str(secret_key).encode()).digest()
    cipher = AES.new(hash_secret_key, AES.MODE_ECB)
    return cipher.encrypt(m).hex()

def generate_primes() -> int:
    p = getStrongPrime(512)
    q = getStrongPrime(512)
    
    while gcd(p*q, (p-1)*(q-1)) != 1:
        print("loop")
        p = getStrongPrime(512)
        q = getStrongPrime(512)

    return p*q

def generate_public_key() -> Tuple[int, int]:
    n = generate_primes()
    k = randint(2, n-1)
    while gcd(k, n) != 1:
        k = randint(2, n-1)
    g = 1 + k * n
    return n, g, k

flag = b"PWNME{xxxxxxxxxxxxxxxxxxxxxxxxx}"

#%%

n = 130480001264795511204952981970554765286628282097708497573805562495761746956689294837477924716000173700265689121058390655726461662172763702188805523675445230642476356316152454104476211773099930843629048798894397653741145611772970364363628025189743819724119397704649989182196725015667676292311250680303497618517
g = 14232999694821698106937459755169111250723143832548091913379257481041382160905011536064172867298828679844798321319150896238739468953330826850323402142301574319504629396273693718919620024174195297927441113170542054761376462382214102358902439525383324742996901035237645136720903186256923046588009251626138008729683922041672060805697738869610571751318652149349473581384089857319209790798013971104266851625853032010411092935478960705260673746033508293802329472778623222171537591292046922903109474029045030942924661333067125642763133098420446959785042615587636015849430889154003912947938463326118557965158805882580597710148
k = 109081848228506024782212502305948797716572300830339785578465230204043919222714279516643240420456408658167645175971167179492414538281767939326117482613367750888391232635306106151999375263906703485783436272382449557941704742019717763385971731987034043089865070488786181508175732060731733665723128263548176110391
A = 10331979810348166693003506393334562363373083416444082955583854323636220335613638441209816437198980825253073980493123573286927762799807446436773117670818921078297923733365129554252727963674496148945815529457095198387555733553703069705181377382893601879633657895337279524071439340411690401699779320407420258592904893010800421041848764790649945309236525529148459624417556599146885803882692326627657181584151248747924080070945415558421472606778565193931117263508570619290441914589981949634553417159683167906276897159926442471600725573380647253372071392282203683205441190912735696337884772579017885457286629133944441076065
B = 4081342267323018166249607688978380665241423816957875747125328810958590656153973787783246867777679461978030117454679495989870502705358238920918102708702013201363687875430336612386215884751792630402395947375495263771248401103245739000962715422458344125251671671250588124240486938525081520695571867300148511333511433839123962435025865462662009339451634433842267524048553313626315201481951251476302835595998914217740184369102003837614515913319042566394680732429410107620067602633793215206219823499602447575406162296590635685877032818801721681953430382920303700518722500790613216329394164889181089201919505288870098353385
enc = "abd9dd2798f4c17b9de4556da160bd42b1a5e3a331b9358ffb11e7c7b3120ed3"



```python
n, g, k = generate_public_key()

a = randint(2, n-1)
b = randint(2, n-1)


A = pow(g, a, n*n)
B = pow(g, b, n*n)

secret_key = pow(B, a, n*n)

print(f"{n = }")
print(f"{g = }")
print(f"{k = }")

print(f"{A = }")
print(f"{B = }")

print(f'enc = "{encrypt(flag, secret_key)}"')
```

Facts:

This looks like the basis of pallier crypto system. Imporant here is that

$(1+x)^n == 1 + nx \mod n^2$ 

$g^a == (1+kn)^a == 1+kna \mod n^2$, we can now continue to modify it until we end up with $na$

In [17]:
k_inv = pow(k,-1,n*n)
k_inv

485767541397027204796694052654312835219571748996540767316709921956403172831859522122502118311434387003774341139146472568399134985773055042831093219277027835741801039391536885727553261740761445202881129545278264008006566096799596061665295866293920482458489953547280208157116763436859922445243684031170812144422179756902952796095692566952251886937860023384176893606423581891500437710825127677998151319978179421920123620814951111948293994414591375809207642512428692208584056583547292406387989881497986260168236970686551578205044902512202980216515078057380316019706270235665045720716374938350027249121333553453024418915

In [18]:
na = ((A - 1) * k_inv) % (n*n)
nb = ((B - 1) * k_inv) % (n*n)
na,nb

(15941053848593865955474772420365230460198990088333118155070289089652222903362178508211659120836658700802585227161552745216260286604097617219280315875926579120785403746798455319081841159920791748108998148946627489840340310818771922723878026051538529741054074821181636644358209004284781528470779535557339161054410765666819037617033093399129017478153392569283276487885443195354419372328953154719844602430566954767116098956113024415124369868639677919269146708798358662513431121228236962626801739810079823243518732431674802145556434174164160872767923406835928576545475011553554967655771112144408833647296244189664973387943,
 4132258024633458317017290712232372442809436119129915721405828053804847228826454121023706907373016914972628135462447072388029710014398576861815273754671134757152543818153284438162334678588439685716219459361977107666978475562226125684498281299474973073921833097592591908565147183507743832071480932931683273370704742244345140280567585841389178001193672401155705173855199411871138453

Now, we have

$n*a \mod n^2 == n*a - n^2*k$ where we can simply divide by n to obtain $a$ because $n$ is on both sides such that we end up with $a \mod n$:)  

In [21]:
a = na//n 
b = nb//n 

A = pow(g, a, n*n)
B = pow(g, b, n*n)

secret_key = pow(B, a, n*n)

In [22]:
hash_secret_key = sha256(str(secret_key).encode()).digest()
cipher = AES.new(hash_secret_key, AES.MODE_ECB)
cipher.decrypt(bytes.fromhex("abd9dd2798f4c17b9de4556da160bd42b1a5e3a331b9358ffb11e7c7b3120ed3"))

b'PWNME{Thi5_1s_H0w_pAl1ier_WorKs}'